In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("C://Users//Abhishek//Downloads//tmdb-movie-metadata//tmdb_5000_movies.csv")
data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [3]:
formatted_data = {} # A consolidated repository of data, a master file. 
available_genres = [] # A list that shows all the different genres available

def findGenres(data):
    # Gathering a list of different genres available in the data
    available_genres = []
    movie_genres = [] 
    copy_data = data.copy() # Making a copy of the data 
    
    for movie_index, movie_data in data.iterrows():
        genres_str = movie_data["genres"]
        # genres is now a string that contains a list which contains a dictionary. This line converts the string into a list
        genres_str = genres_str.strip('][')
        
        brace_positions = [] # A list that stores lists of start and end positions of all genres (brace positions).
        brace_position = [] # A list that stores start and end position of a single genre
        for index, letter in enumerate(genres_str): 
            if letter == '{':
                brace_position.append(index)
            if letter == '}':
                brace_position.append(index)
                brace_positions.append(brace_position)
                brace_position = [] # Resetting genre_positions to capture other dictionaries
        
        movie_genres = []
        
        for position in brace_positions:
            temp = genres_str[position[0]:position[1]+1]
            genre_dict = json.loads(temp)
            genre = genre_dict["name"]
            if genre not in available_genres:
                available_genres.append(genre)
            movie_genres.append(genre)
        # movie_genres = pd.Series(movie_genres)
        copy_data.at[movie_index, "genres"] =  movie_genres
      
    return available_genres, copy_data

available_genres, genre_formatted_data = findGenres(data)
available_genres 

['Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'Crime',
 'Drama',
 'Thriller',
 'Animation',
 'Family',
 'Western',
 'Comedy',
 'Romance',
 'Horror',
 'Mystery',
 'History',
 'War',
 'Music',
 'Documentary',
 'Foreign',
 'TV Movie']

In [4]:
genre_movie_mapper = {} # A mapper that maps different movies to the genres it belongs to. 

# Fill all genres with empty lists in genre_movie_mapper

for genre in available_genres:
    genre_movie_mapper[genre] = [] 

genre_movie_mapper

def map_movies_to_genres(data, genre_movie_mapper): 

    for movie_index, movie_data in data.iterrows(): 
        genres = movie_data["genres"]
        movie_title = movie_data["original_title"]
        
        for genre in genres:
            movie_list = genre_movie_mapper[genre]
            movie_list.append(movie_title)
            genre_movie_mapper[genre] = movie_list
    return genre_movie_mapper

genre_movie_mapper = map_movies_to_genres(genre_formatted_data, genre_movie_mapper)

for genre in genre_movie_mapper.keys():
    print(len(genre_movie_mapper[genre]))


1154
790
424
535
696
2297
1274
234
513
82
1722
894
519
348
197
144
185
110
34
8


In [5]:
def findKeywords(data):
    # Gathering a list of different keywords available in the data
    available_keywords = []
    movie_keywords = [] 
    copy_data = data.copy() # Making a copy of the data 
    
    for movie_index, movie_data in data.iterrows():
        keywords_str = movie_data["keywords"]
        # keywords_str is now a string that contains a list which contains a dictionary. This line converts the string into a list
        keywords_str = keywords_str.strip('][')
      
        brace_positions = [] # A list that stores lists of start and end positions of all keywords (brace positions).
        brace_position = [] # A list that stores start and end position of a single kkeywords
        for index, letter in enumerate(keywords_str): 
            if letter == '{':
                brace_position.append(index)
            if letter == '}':
                brace_position.append(index)
                brace_positions.append(brace_position)
                brace_position = [] # Resetting keyword_positions to capture other dictionaries
        movie_keywords = []
        
        for position in brace_positions:
            temp = keywords_str[position[0]:position[1]+1]
            keyword_dict = json.loads(temp)
            keyword = keyword_dict["name"]
            if keyword not in available_keywords:
                available_keywords.append(keyword)
            movie_keywords.append(keyword)
        copy_data.at[movie_index, "keywords"] =  movie_keywords
        
    return available_keywords, copy_data

available_keywords, keywords_formatted_data = findKeywords(genre_formatted_data)

In [6]:
sorted(available_keywords)

['15th century',
 '16th century',
 '17th century',
 '18th century',
 '1910s',
 '1920s',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '19th century',
 '2000 ad',
 '20th century',
 '21st century',
 '25th century',
 '2nd century',
 '3d',
 'abandoned house',
 'abandoned mine',
 'abduction',
 'abolitionist',
 'aboriginal',
 'aborigine',
 'abortion',
 'abraham lincoln',
 'absorbing power',
 'absurdism',
 'abuse',
 'abusive father',
 'abusive husband',
 'abusive marriage',
 'acapella',
 'accepting death',
 'accident',
 'accidental love',
 'accountant',
 'acid',
 'acting',
 'action',
 'action hero',
 'action heroine',
 'activist',
 'actor',
 'actors',
 'actress',
 'adaptation',
 'addict',
 'addicted',
 'addiction',
 'admiration',
 'admissions',
 'adolescence',
 'adolescent boy',
 'adolf hitler',
 'adopted child',
 'adoption',
 'adoptive father',
 'adoptive mother',
 'adrenalin',
 'adrenaline junkie',
 'adult animation',
 'adult as a child',
 'adult humor',
 'adultery'

In [7]:
keyword_movie_mapper = {} # A mapper that maps different movies & keywords. 

# Fill all keywords with empty lists in keyword_movie_mapper

for keyword in available_keywords:
    keyword_movie_mapper[keyword] = [] 

keyword_movie_mapper

def map_movies_to_keywords(data, keyword_movie_mapper): 

    for movie_index, movie_data in data.iterrows(): 
        keywords = movie_data["keywords"]
        movie_title = movie_data["original_title"]
        
        for keyword in keywords:
            movie_list = keyword_movie_mapper[keyword]
            movie_list.append(movie_title)
            keyword_movie_mapper[keyword] = movie_list
    return keyword_movie_mapper

keyword_movie_mapper = map_movies_to_keywords(keywords_formatted_data, keyword_movie_mapper)
keyword_movie_mapper["1970s"]
keywords_formatted_data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[Action, Crime, Thriller]",NaN,9367,"[united states–mexico barrier, legs, arms, pap...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[Comedy, Romance]",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[Comedy, Drama, Romance, TV Movie]",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[date, love at first sight, narration, investi...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,[],http://shanghaicalling.com/,126186,[],en,Shanghai Calling,When ambitious Ne

In [8]:
def findProductionCompanies(data):
    # Gathering a list of different production companies  available in the data
    available_productionCompanies = []
    movie_productionCompanies = [] 
    copy_data = data.copy() # Making a copy of the data 
    
    for movie_index, movie_data in data.iterrows():
        productionCompanies_str = movie_data["production_companies"]
        # productionCompanies_str is now a string that contains a list which contains a dictionary. This line converts the string into a list
        productionCompanies_str = productionCompanies_str.strip('][')        
        brace_positions = [] # A list that stores lists of start and end positions of all productionCompanies (brace positions).
        brace_position = [] # A list that stores start and end position of a single kproductionCompanies
        for index, letter in enumerate(productionCompanies_str): 
            if letter == '{':
                brace_position.append(index)
            if letter == '}':
                brace_position.append(index)
                brace_positions.append(brace_position)
                brace_position = [] # Resetting productionCompany_positions to capture other dictionaries
        movie_productionCompanies = []
        
        for position in brace_positions:
            temp = productionCompanies_str[position[0]:position[1]+1]
            productionCompany_dict = json.loads(temp)
            productionCompany = productionCompany_dict["name"]
            if productionCompany not in available_productionCompanies:
                available_productionCompanies.append(productionCompany)
            movie_productionCompanies.append(productionCompany)
        copy_data.at[movie_index, "production_companies"] =  movie_productionCompanies
        
    return available_productionCompanies, copy_data

available_productionCompanies, productionCompanies_formatted_data = findProductionCompanies(keywords_formatted_data)
available_productionCompanies

['Ingenious Film Partners',
 'Twentieth Century Fox Film Corporation',
 'Dune Entertainment',
 'Lightstorm Entertainment',
 'Walt Disney Pictures',
 'Jerry Bruckheimer Films',
 'Second Mate Productions',
 'Columbia Pictures',
 'Danjaq',
 'B24',
 'Legendary Pictures',
 'Warner Bros.',
 'DC Entertainment',
 'Syncopy',
 'Laura Ziskin Productions',
 'Marvel Enterprises',
 'Walt Disney Animation Studios',
 'Marvel Studios',
 'Prime Focus',
 'Revolution Sun Studios',
 'Heyday Films',
 'DC Comics',
 'Atlas Entertainment',
 'Cruel & Unusual Films',
 'RatPac-Dune Entertainment',
 'Bad Hat Harry Productions',
 'Eon Productions',
 'Infinitum Nihil',
 'Silver Bullet Productions (II)',
 'Blind Wink Productions',
 'Classic Media',
 'Cruel and Unusual Films',
 'Walt Disney',
 'Walden Media',
 'Stillking Films',
 'Ozumi Films',
 'Propeler',
 'Silverbell Films',
 'Paramount Pictures',
 'Moving Picture Company (MPC)',
 'Amblin Entertainment',
 'Media Magik Entertainment',
 'Imagenation Abu Dhabi FZ',
 '

In [9]:
productionCompany_movie_mapper = {} # A mapper that maps different movies to the productionCompanies it belongs to. 

# Fill all productionCompanies with empty lists in productionCompany_movie_mapper

for productionCompany in available_productionCompanies:
    productionCompany_movie_mapper[productionCompany] = [] 

productionCompany_movie_mapper

def map_movies_to_productionCompanies(data, productionCompany_movie_mapper): 

    for movie_index, movie_data in data.iterrows(): 
        productionCompanies = movie_data["production_companies"]
        movie_title = movie_data["original_title"]
        
        for productionCompany in productionCompanies:
            movie_list = productionCompany_movie_mapper[productionCompany]
            movie_list.append(movie_title)
            productionCompany_movie_mapper[productionCompany] = movie_list
    return productionCompany_movie_mapper

productionCompany_movie_mapper = map_movies_to_productionCompanies(productionCompanies_formatted_data, productionCompany_movie_mapper)
productionCompany_movie_mapper["Contrafilm"]

['Journey 2: The Mysterious Island',
 'After the Sunset',
 'The Rite',
 'The Number 23',
 "What's Your Number?",
 'Choke']

In [10]:
def findProductionCountries(data):
    # Gathering a list of different production companies  available in the data
    available_productionCountries = []
    movie_productionCountries = [] 
    copy_data = data.copy() # Making a copy of the data 
    
    for movie_index, movie_data in data.iterrows():
        productionCountries_str = movie_data["production_countries"]
        # productionCountries_str is now a string that contains a list which contains a dictionary. This line converts the string into a list
        productionCountries_str = productionCountries_str.strip('][')        
        brace_positions = [] # A list that stores lists of start and end positions of all productionCountries (brace positions).
        brace_position = [] # A list that stores start and end position of a single kproductionCountries
        for index, letter in enumerate(productionCountries_str): 
            if letter == '{':
                brace_position.append(index)
            if letter == '}':
                brace_position.append(index)
                brace_positions.append(brace_position)
                brace_position = [] # Resetting productionCountry_positions to capture other dictionaries
        movie_productionCountries = []
        
        for position in brace_positions:
            temp = productionCountries_str[position[0]:position[1]+1]
            productionCountry_dict = json.loads(temp)
            productionCountry = productionCountry_dict["name"]
            if productionCountry not in available_productionCountries:
                available_productionCountries.append(productionCountry)
            movie_productionCountries.append(productionCountry)
        copy_data.at[movie_index, "production_countries"] =  movie_productionCountries
        
    return available_productionCountries, copy_data

available_productionCountries, productionCountries_formatted_data = findProductionCountries(productionCompanies_formatted_data)
available_productionCountries

['United States of America',
 'United Kingdom',
 'Jamaica',
 'Bahamas',
 'Dominica',
 'Czech Republic',
 'Poland',
 'Slovenia',
 'New Zealand',
 'Germany',
 'China',
 'Canada',
 'Italy',
 'Japan',
 'Malta',
 'Australia',
 'France',
 'Belgium',
 'India',
 'Netherlands',
 'Spain',
 'United Arab Emirates',
 'Hong Kong',
 'Taiwan',
 'Ireland',
 'Morocco',
 'Hungary',
 'Singapore',
 'Norway',
 'Sweden',
 'South Africa',
 'Russia',
 'Romania',
 'Mexico',
 'Monaco',
 'Switzerland',
 'Pakistan',
 'Malaysia',
 'Finland',
 'Iceland',
 'Denmark',
 'Tunisia',
 'Philippines',
 'Bulgaria',
 'South Korea',
 'Brazil',
 'Peru',
 'Luxembourg',
 'Bosnia and Herzegovina',
 'Kazakhstan',
 'Portugal',
 'Aruba',
 'Libyan Arab Jamahiriya',
 'Serbia',
 'Ukraine',
 'Chile',
 'Argentina',
 'Panama',
 'Austria',
 'Greece',
 'Lithuania',
 'Cambodia',
 'Thailand',
 'Slovakia',
 'Israel',
 'Fiji',
 'Serbia and Montenegro',
 'Turkey',
 'Nigeria',
 'Cyprus',
 'Jordan',
 'Bolivia',
 'Ecuador',
 'Colombia',
 'Egypt',
 '

In [11]:
productionCountry_movie_mapper = {} # A mapper that maps different movies to the productionCountries it belongs to. 

# Fill all productionCountries with empty lists in productionCountry_movie_mapper

for productionCountry in available_productionCountries:
    productionCountry_movie_mapper[productionCountry] = [] 

productionCountry_movie_mapper

def map_movies_to_productionCountries(data, productionCountry_movie_mapper): 

    for movie_index, movie_data in data.iterrows(): 
        productionCountries = movie_data["production_countries"]
        movie_title = movie_data["original_title"]
        
        for productionCountry in productionCountries:
            movie_list = productionCountry_movie_mapper[productionCountry]
            movie_list.append(movie_title)
            productionCountry_movie_mapper[productionCountry] = movie_list
    return productionCountry_movie_mapper

productionCountry_movie_mapper = map_movies_to_productionCountries(productionCountries_formatted_data, productionCountry_movie_mapper)
productionCountry_movie_mapper["India"]

['The Lovers',
 'The BFG',
 'Real Steel',
 'Dredd',
 'Walking With Dinosaurs',
 'Don Gato: El inicio de la pandilla',
 "Legends of Oz: Dorothy's Return",
 'Lincoln',
 'The Happening',
 'Street Fighter: The Legend of Chun-Li',
 '1947: Earth',
 'Bridge of Spies',
 'Baahubali: The Beginning',
 'The Help',
 'Gandhi',
 'The Hundred-Foot Journey',
 'Alpha and Omega',
 'ராமானுஜன்',
 'Slumdog Millionaire',
 'I Think I Love My Wife',
 'कभी अलविदा ना कहना',
 'Housefull',
 'Kites',
 'Jab Tak Hai Jaan',
 'The Namesake',
 'My Name Is Khan',
 'Yeh Jawaani Hai Deewani',
 'Ta Ra Rum Pum',
 'Veer-Zaara',
 'Stiff Upper Lips',
 'Bend It Like Beckham',
 'Rang De Basanti',
 'Gandhi, My Father',
 'Fugly',
 'Airlift',
 'Paa',
 'Kama Sutra - A Tale of Love',
 'Lage Raho Munna Bhai',
 'Sardaarji',
 'Brick Lane',
 'The Other End of the Line',
 'Roadside Romeo',
 'एबीसीडी',
 'வாலு',
 'Dabba',
 'दिल जो भी कहे',
 "Neal 'n' Nikki",
 'Monsoon Wedding',
 'Rocket Singh: Salesman of the Year',
 'Fiza',
 'Chocolate: Dee

In [12]:
'''Most Common Keywords'''

def findMostCommonKeywords(keyword_movie_mapper):
    keyword_movie_length_mapper = {} 
    
    for keyword, movies in keyword_movie_mapper.items():
        keyword_movie_length_mapper[keyword] = len(movies)
    
    keyword_movie_length_mapper = {k: v for k, v in sorted(keyword_movie_length_mapper.items(), key=lambda item: item[1], reverse=True)} # Sorting keyword_movie_length_mapper by number of movies. 
 
    return keyword_movie_length_mapper


keyword_movie_length_mapper = findMostCommonKeywords(keyword_movie_mapper)
keyword_movie_length_mapper


'''Movie with most Keywords'''

def findMoviesWithMostKeywords(data): 
    movie_keyword_counter = 0
    moviesWithMostKeywords = ""
    mostKeywords = []
    keywords_length_list = []
    
    for movie_index, movie_data in data.iterrows():
        keywords = movie_data["keywords"]
        keywords_length = len(keywords)
        keywords_length_list.append(keywords_length)
        if keywords_length > movie_keyword_counter:
            movie_keyword_counter = keywords_length
            moviesWithMostKeywords = movie_data["original_title"]
            mostKeywords = keywords
    
    ''' Finding average count of keyword occurrences '''
    len_keywords = len(keywords_length_list)
    sum_keywords = sum(keywords_length_list)
    
    print(sum_keywords, len_keywords)
    
    avg_keywords_count = sum_keywords/len_keywords
    
    return moviesWithMostKeywords, mostKeywords, avg_keywords_count


moviesWithMostKeywords, mostKeywords, avg_keywords_count  = findMoviesWithMostKeywords(keywords_formatted_data)

print(moviesWithMostKeywords)
print(mostKeywords)
print(len(mostKeywords))
print(avg_keywords_count)


''' Analysis: '''
# The keyword 'woman director' occurs the most times across all movies with 'independent film' coming after that. 

36194 4803
Brooklyn's Finest
['male nudity', 'female nudity', 'tattoo', 'gambling', 'corruption', 'father son relationship', 'prostitute', 'robbery', 'detective', 'police brutality', 'wife husband relationship', 'card game', 'desperation', 'undercover', 'kidnapping', 'drug dealer', 'retirement', 'death of a friend', 'asthma', 'police', 'fellatio', 'blood splatter', 'revenge', 'deception', 'murder', 'betrayal', 'priest', 'breast', 'bag of money', 'shootout', 'blood on shirt', 'shot in the back', 'undercover cop', 'blood', 'suicidal', 'f word', 'shot to death', 'punched in the face', 'redemption', 'police officer killed', 'hospital', 'dirty cop', 'police corruption', 'pistol', 'new york city', 'church', 'violence', 'strangulation', 'foot chase', 'police detective', 'father daughter relationship', 'police station', 'held at gunpoint', 'catholic', 'guilt', 'brooklyn new york city', 'narcotics cop', 'nypd', 'missing person', 'racial slur', 'swat team', 'ensemble cast', 'pregnant wife', 'con

' Analysis: '

In [13]:
'''Movie with most Keywords'''

def findMoviesWithMostKeywords(data): 
    movie_keyword_counter = 0
    moviesWithMostKeywords = ""
    mostKeywords = []
    keywords_length_list = []
    
    for movie_index, movie_data in data.iterrows():
        keywords = movie_data["keywords"]
        keywords_length = len(keywords)
        keywords_length_list.append(keywords_length)
        if keywords_length > movie_keyword_counter:
            movie_keyword_counter = keywords_length
            moviesWithMostKeywords = movie_data["original_title"]
            mostKeywords = keywords
    
    ''' Finding average count of keyword occurrences '''
    len_keywords = len(keywords_length_list)
    sum_keywords = sum(keywords_length_list)
    
    print(sum_keywords, len_keywords)
    
    avg_keywords_count = sum_keywords/len_keywords
    
    return moviesWithMostKeywords, mostKeywords, avg_keywords_count


moviesWithMostKeywords, mostKeywords, avg_keywords_count  = findMoviesWithMostKeywords(productionCountries_formatted_data)

print(moviesWithMostKeywords)
print(mostKeywords)
print(len(mostKeywords))
print(avg_keywords_count)

''' Analysis: '''
# The movie with the most number of keywords is Brooklyn's Finest with 97 keywords. 
# A total of 36194 keywords exist for 4803 movies with an average of 7.53 ~ 8 keywords. 

36194 4803
Brooklyn's Finest
['male nudity', 'female nudity', 'tattoo', 'gambling', 'corruption', 'father son relationship', 'prostitute', 'robbery', 'detective', 'police brutality', 'wife husband relationship', 'card game', 'desperation', 'undercover', 'kidnapping', 'drug dealer', 'retirement', 'death of a friend', 'asthma', 'police', 'fellatio', 'blood splatter', 'revenge', 'deception', 'murder', 'betrayal', 'priest', 'breast', 'bag of money', 'shootout', 'blood on shirt', 'shot in the back', 'undercover cop', 'blood', 'suicidal', 'f word', 'shot to death', 'punched in the face', 'redemption', 'police officer killed', 'hospital', 'dirty cop', 'police corruption', 'pistol', 'new york city', 'church', 'violence', 'strangulation', 'foot chase', 'police detective', 'father daughter relationship', 'police station', 'held at gunpoint', 'catholic', 'guilt', 'brooklyn new york city', 'narcotics cop', 'nypd', 'missing person', 'racial slur', 'swat team', 'ensemble cast', 'pregnant wife', 'con

' Analysis: '

In [14]:
'''Building a Movie Details Mapper which maps the movie with all the relevant information'''

def buildMovieDetailsMapper(data):
    movie_details_mapper = {}
    
    for movie_index, movie_data in data.iterrows():
        movie_title = movie_data["original_title"]
        
        movie_data_dict = {}
        
        movie_data_dict["budget"] = movie_data["budget"]
        movie_data_dict["genres"] = movie_data["genres"]
        movie_data_dict["keywords"] = movie_data["keywords"]
        movie_data_dict["popularity"] = movie_data["popularity"]
        movie_data_dict["production_companies"] = movie_data["production_companies"]
        movie_data_dict["production_countries"] = movie_data["production_countries"]
        movie_data_dict["revenue"] = movie_data["revenue"]
        movie_data_dict["vote_average"] = movie_data["vote_average"]
        
        movie_details_mapper[movie_title] = movie_data_dict
    return movie_details_mapper 

movie_details_mapper = buildMovieDetailsMapper(productionCountries_formatted_data) # Returns a dictionary that contains movie_names as keys and a list which again contains a dictionary of relevant information for the movie

In [15]:
movie_details_mapper

{'Avatar': {'budget': 237000000,
  'genres': ['Action', 'Adventure', 'Fantasy', 'Science Fiction'],
  'keywords': ['culture clash',
   'future',
   'space war',
   'space colony',
   'society',
   'space travel',
   'futuristic',
   'romance',
   'space',
   'alien',
   'tribe',
   'alien planet',
   'cgi',
   'marine',
   'soldier',
   'battle',
   'love affair',
   'anti war',
   'power relations',
   'mind and soul',
   '3d'],
  'popularity': 150.437577,
  'production_companies': ['Ingenious Film Partners',
   'Twentieth Century Fox Film Corporation',
   'Dune Entertainment',
   'Lightstorm Entertainment'],
  'production_countries': ['United States of America', 'United Kingdom'],
  'revenue': 2787965087,
  'vote_average': 7.2},
 "Pirates of the Caribbean: At World's End": {'budget': 300000000,
  'genres': ['Adventure', 'Fantasy', 'Action'],
  'keywords': ['ocean',
   'drug abuse',
   'exotic island',
   'east india trading company',
   "love of one's life",
   'traitor',
   'shipwre

In [16]:
''' Keyword Movie Budget Movie Revenue comparison'''


' Keyword Movie Budget Movie Revenue comparison'